### Problem Statement:

To verify if the charges levied by their delivery partners per order are correct

#### Approach:

- The data fiels were read in as DataFrame for both LHS (X Company data) & RHS (Courier Company Data) using pandas
- The sanity check of data was done to ensure data was consistent
- Initiated the process with Order Report of X company, by merging it with the SKU master weight was obtained for SKU
- The weight was then converted to KG from G and then weight was groupedby OrderID
-	Then further merged the processed data with courier_invoice to obtain more details on order.
-	To bring in a chain between the Courier companies data and X the zone of X was obtained from warehouse_pincode dataframe of X company.
-	Based on the available information the Type of Shipment and Zone using f-string and conditional looping the rate applicable to the order was obtained in array
-	The obtained array values were then mapped to the data frame and expected charges of x was calculated.
-	Then the difference in rate between X and Courier was calculated
-	From which we had all the required information for Output data 1 and the same was constructed
-	Then a summary table was created by categorising the orders to Correct, Overcharged and Undercharged
-	This was the summary table extract from the data the Output data 2
-	Finally, both the dataframes of Output data 1 & 2 were written to storage as csv files using pandas

### Importing Packages

In [1]:
import pandas as pd

### Importing Data

#### LHS Data (X's internal data spread across three reports)

In [2]:
# Order Report - Orderids, SKU's (Stock keeping Units) and order quantity for each order
X_order_report = pd.read_excel("C:/Users/pjpra/OneDrive/Desktop/My_Learnings/Interview/Cointab/Cointab Data Analyst - Challenge/Company X - Order Report.xlsx")

# SKU Master - Gross weight of each product
X_sku_master = pd.read_excel("C:/Users/pjpra/OneDrive/Desktop/My_Learnings/Interview/Cointab/Cointab Data Analyst - Challenge/Company X - SKU Master.xlsx")

# All India Pincode for warehouse/customer pincode and corresponding zone
X_pin_code_z = pd.read_excel("C:/Users/pjpra/OneDrive/Desktop/My_Learnings/Interview/Cointab/Cointab Data Analyst - Challenge/Company X - Pincode Zones.xlsx")

#### Sanity Check of LHS Data

In [3]:
print(X_order_report.columns)
print(X_order_report.isnull().sum())
print(X_order_report.SKU.nunique())
print(X_order_report['Order Qty'].unique())
print(X_order_report.shape)
print(X_order_report.head(2))
print(X_order_report['ExternOrderNo'].nunique(), X_order_report['ExternOrderNo'].count())

Index(['ExternOrderNo', 'SKU', 'Order Qty'], dtype='object')
ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64
65
[1. 2. 4. 3. 8. 6.]
(400, 3)
   ExternOrderNo            SKU  Order Qty
0     2001827036  8904223818706        1.0
1     2001827036  8904223819093        1.0
124 400


In [4]:
X_sku_master.head(2)
print(X_sku_master.columns)
print(X_sku_master.SKU.nunique())
print(X_sku_master['Weight (g)'].unique())
print(X_sku_master['Weight (g)'].nunique())
print(X_sku_master.shape)
X_sku_master.isnull().sum()

Index(['SKU', 'Weight (g)'], dtype='object')
65
[210 165 113  65 120 102 170 350 115 150 137 240 121 127 160 232 100 490
 133 112 110  30 140 290 552 500 250 130 600 300  10]
31
(66, 2)


SKU           0
Weight (g)    0
dtype: int64

In [5]:
X_pin_code_z.head(2)
print(X_pin_code_z.columns)
print(X_pin_code_z['Warehouse Pincode'].nunique())
print(X_pin_code_z['Customer Pincode'].nunique())
print(X_pin_code_z['Zone'].unique())
print(X_pin_code_z.shape)
X_pin_code_z.isnull().sum()

Index(['Warehouse Pincode', 'Customer Pincode', 'Zone'], dtype='object')
1
108
['d' 'b' 'e']
(124, 3)


Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

In [6]:
X_pin_code_z.rename(columns = {'Zone':'Zone by X'}, inplace = True)

#### RHS Data (Courier company invoice in CSV file)

In [7]:
Courier_invoice = pd.read_excel("C:/Users/pjpra/OneDrive/Desktop/My_Learnings/Interview/Cointab/Cointab Data Analyst - Challenge/Courier Company - Invoice.xlsx")
Courier_rates = pd.read_excel("C:/Users/pjpra/OneDrive/Desktop/My_Learnings/Interview/Cointab/Cointab Data Analyst - Challenge/Courier Company - Rates.xlsx")

#### Sanity Check of RHS data

In [8]:
print(Courier_invoice.isnull().sum())
print("\nColumns:\n",Courier_invoice.columns)

# Check if the unique fields are unique
print("\nAWB Code:\n",Courier_invoice['AWB Code'].nunique(), Courier_invoice['AWB Code'].count())
print("\nOrder ID:\n",Courier_invoice['Order ID'].nunique(), Courier_invoice['Order ID'].count())

print("\nZone\n",Courier_invoice['Zone'].unique())
print("\nType of Shipment:\n", Courier_invoice['Type of Shipment'].unique())

print("\nBilling Amount:\n",Courier_invoice['Billing Amount (Rs.)'].unique())
print("\nCharged Weight:\n",Courier_invoice['Charged Weight'].unique())

AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Columns:
 Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')

AWB Code:
 124 124

Order ID:
 124 124

Zone
 ['d' 'b' 'e']

Type of Shipment:
 ['Forward charges' 'Forward and RTO charges']

Billing Amount:
 [135.   90.2 224.6  61.3  45.4  89.6  33.  107.3 172.8 102.3 213.5 258.9
 151.1 345.  174.5 179.8 269.4  86.7 117.9 403.8]

Charged Weight:
 [1.3  1.   2.5  0.15 1.15 0.5  0.79 0.72 1.08 1.28 0.2  0.86 1.2  0.7
 0.6  0.99 0.8  1.6  1.13 2.92 0.68 0.71 0.78 1.27 0.69 1.16 2.86 1.35
 1.64 0.67 2.   1.5  3.   1.7  0.77 0.76 0.59 2.94 0.61 2.1  1.1  0.3
 1.02 2.28 0.74 4.13 0.73 1.04 1.63 2.47 0.82 0.66 1.86 2.27]


In [9]:
Courier_rates.head()

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [10]:
# For construction of output data 1
sample_df = pd.read_excel("C:/Users/pjpra/OneDrive/Desktop/My_Learnings/Interview/Cointab/Cointab Data Analyst - Challenge/Expected_Result.xlsx", sheet_name='Calculations')
Output_Data1 = pd.DataFrame(columns=sample_df.columns)

### Data Consolidation

In [11]:
# Merge X_order_report & X_sku_master
merge1 = pd.merge(X_order_report,X_sku_master,on='SKU')

In [12]:
# Obtain weight from G to KG
merge1['Total_weight (KG)'] = (merge1['Order Qty'] * merge1['Weight (g)']) / 1000
merge1.head(2)

,ExternOrderNo,SKU,Order Qty,Weight (g),Total_weight (KG)
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127


In [13]:
# Group Data by Order_ID
gp_byorder = merge1.groupby('ExternOrderNo')['Total_weight (KG)'].sum().reset_index()
gp_byorder.head(1)
gp_byorder.dtypes
# Find Weight_slab
slab_scale = {0: 0, 0.001: 0.5, 0.501: 1, 1.0 : 1.5, 1.51 : 2, 2.0 : 2.5}
gp_byorder['Weight_slab'] = gp_byorder['Total_weight (KG)'].apply(lambda x: slab_scale[max(k for k in slab_scale.keys() if k <= x)] if x > 0 else 0)
gp_byorder = gp_byorder.rename(columns = {'ExternOrderNo':'Order ID','Total_weight (KG)':'Total weight by x (kg)','Weight_slab':'Weight slab by X'})

In [14]:
# Value Check
gp_byorder.count()

Order ID                  124
Total weight by x (kg)    124
Weight slab by X          124
dtype: int64

In [15]:
# Merge gp_byorder to courier_invoice
merge2 = pd.merge(gp_byorder, Courier_invoice,on = 'Order ID')
merge2.isnull().sum()
merge2['Order ID'].nunique()
merge2.shape
merge2.columns
merge2.count()
merge2['Type of Shipment'].unique()

array(['Forward charges', 'Forward and RTO charges'], dtype=object)

In [16]:
# Merge Merge2 & X_pin_code_z to add zone by x
merge3 = pd.merge(merge2,X_pin_code_z, on = ['Warehouse Pincode','Customer Pincode'])
slab_scale = {0: 0, 0.001: 0.5, 0.501: 1, 1.0 : 1.5, 1.51 : 2, 2.0 : 2.5}
merge3['Weight slab charged by Courier Company'] = (merge3['Charged Weight']).apply(lambda x: slab_scale[max(k for k in slab_scale.keys() if k <= x)] if x > 0 else 0)
merge3.drop_duplicates(inplace = True)
merge3.head(4)

,Order ID,Total weight by x (kg),Weight slab by X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Zone by X,Weight slab charged by Courier Company
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,2.5
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,1.0
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,d,1.0
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,d,1.5


#### Calculation of Expected Charges as per X

In [17]:
# Code to parse values based on each order in order table to base cost table
calc_cost = merge3
tempx = Courier_rates.melt(var_name='description', value_name='value')
tempx.description.unique()
fwd_fixed,fwd_additional,rto_fixed,rto_additional = [],[],[],[]
for i in range(len(calc_cost)):
    if (calc_cost.iloc[i,8] == 'Forward charges') & (calc_cost.iloc[i,2] <= 0.5):
        cost1 = f'fwd_{calc_cost.iloc[i,10]}_fixed'
        fwd_fixed.append(float(tempx.loc[tempx['description'] == cost1 , 'value'].values))
        fwd_additional.append(0)
        rto_fixed.append(0)
        rto_additional.append(0)
    elif (calc_cost.iloc[i,8] == 'Forward charges') & (calc_cost.iloc[i,2] > 0.5):
        cost1 = f'fwd_{calc_cost.iloc[i,10]}_fixed'
        cost2 = f'fwd_{calc_cost.iloc[i,10]}_additional'
        fwd_fixed.append(float(tempx.loc[tempx['description'] == cost1 , 'value'].values))
        fwd_additional.append(float((tempx.loc[tempx['description'] == cost2 , 'value'].values)*(calc_cost.iloc[i,2] - 0.5)))
        rto_fixed.append(0)
        rto_additional.append(0)
    elif (calc_cost.iloc[i,8] == 'Forward and RTO charges') & (calc_cost.iloc[i,2] < 0.5):
        cost1 = f'fwd_{calc_cost.iloc[i,10]}_fixed'
        cost3 = f'rto_{calc_cost.iloc[i,10]}_fixed'
        fwd_fixed.append(float(tempx.loc[tempx['description'] == cost1 , 'value'].values))
        fwd_additional.append(0)
        rto_fixed.append(float(tempx.loc[tempx['description'] == cost3 , 'value'].values))
        rto_additional.append(0)
    else:
        cost1 = f'fwd_{calc_cost.iloc[i,10]}_fixed'
        cost2 = f'fwd_{calc_cost.iloc[i,10]}_additional'
        cost3 = f'rto_{calc_cost.iloc[i,10]}_fixed'
        cost4 = f'rto_{calc_cost.iloc[i,10]}_additional'
        fwd_fixed.append(float(tempx.loc[tempx['description'] == cost1 , 'value'].values))
        fwd_additional.append(float((tempx.loc[tempx['description'] == cost2 , 'value'].values)*(calc_cost.iloc[i,2] - 0.5)))
        rto_fixed.append(float(tempx.loc[tempx['description'] == cost3 , 'value'].values))
        rto_additional.append(float((tempx.loc[tempx['description'] == cost4 , 'value'].values)*(calc_cost.iloc[i,2] - 0.5)))

In [18]:
# Creation of columns with extracted cost information of each order
calc_cost2 = calc_cost.drop(calc_cost.iloc[:,1:], axis =1)
calc_cost2['FWD Cost fixed'] = fwd_fixed
calc_cost2['FWD Cost additional'] = fwd_additional
calc_cost2['RTO Cost fixed'] = rto_fixed
calc_cost2['RTO Cost additional'] = rto_additional
calc_cost2['Expected charges as per X (Rs.)'] = calc_cost2['FWD Cost fixed'] + calc_cost2['FWD Cost additional'] + calc_cost2['RTO Cost fixed'] + calc_cost2['RTO Cost additional']

In [19]:
# Inclusion of the charges as per x to main work table
temp1 = calc_cost2[['Order ID','Expected charges as per X (Rs.)']]
merge4 = pd.merge(merge3, temp1, on = 'Order ID')
merge4.head(2)

,Order ID,Total weight by x (kg),Weight slab by X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Zone by X,Weight slab charged by Courier Company,Expected charges as per X (Rs.)
0,2001806210,0.22,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,2.5,33.0
1,2001806226,0.48,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,1.0,45.4


In [20]:
# Cost Difference between X and Courier Company
merge4['Difference Between Expected Charges and Billed Charges (Rs.)'] = merge4['Billing Amount (Rs.)'] - merge4['Expected charges as per X (Rs.)']
merge4.head()

,Order ID,Total weight by x (kg),Weight slab by X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Zone by X,Weight slab charged by Courier Company,Expected charges as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,b,2.5,33.0,141.5
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,d,1.0,45.4,44.8
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,d,1.0,45.4,44.8
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,d,1.5,90.2,44.8
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,b,1.0,33.0,28.3


#### Output 1: Resultant Data Creation

In [21]:
Output_Data1['Order ID'] = merge4['Order ID']
Output_Data1['AWB Number'] = merge4['AWB Code']
Output_Data1['Total weight as per X (KG)'] = merge4['Total weight by x (kg)']
Output_Data1['Weight slab as per X (KG)'] = merge4['Weight slab by X']
Output_Data1['Total weight as per Courier Company (KG)'] = merge4['Charged Weight']
Output_Data1['Weight slab charged by Courier Company (KG)'] = merge4['Weight slab charged by Courier Company']
Output_Data1['Delivery Zone as per X'] = merge4['Zone by X']
Output_Data1['Delivery Zone charged by Courier Company'] = merge4['Zone']
Output_Data1['Expected Charge as per X (Rs.)'] = merge4['Expected charges as per X (Rs.)']
Output_Data1['Charges Billed by Courier Company (Rs.) '] = merge4['Billing Amount (Rs.)']
Output_Data1['Difference Between Expected Charges and Billed Charges (Rs.)'] = merge4['Difference Between Expected Charges and Billed Charges (Rs.)']
Output_Data1.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,2.5,b,b,33.0,174.5,141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.4,90.2,44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.4,90.2,44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,90.2,135.0,44.8
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.0,61.3,28.3


#### Output 2: Summary Table Creation

In [22]:
# Funtion to classify if the field is 
def get_charge_diff_sign(x):
    if x == 0:
        return 'Correct'
    elif x < 0:
        return 'Overcharged'
    else:
        return 'Undercharged'

merge4['charge_diff'] = merge4['Difference Between Expected Charges and Billed Charges (Rs.)'].apply(get_charge_diff_sign)

In [23]:
Summary_table = merge4.groupby('charge_diff').agg({'Order ID': 'count', 'Billing Amount (Rs.)': 'sum'})
Summary_table = Summary_table.rename(columns={'Order ID': 'Total orders', 'Billing Amount (Rs.)': 'Total amount'})
Summary_table

,Total orders,Total amount
charge_diff,,
Correct,7,321.3
Overcharged,7,443.3
Undercharged,110,12883.6


#### To CSV (Output 1 & 2)

In [25]:
Output_Data1.to_csv('Output_Data1.csv', index = False)
Summary_table.to_csv('Output_Data2.csv')